# **Calibration of a complete Mueller polarimeter**
*Author*: @ mariajloperaa

## This notebook contains all the elements to calibrate and use the automatic Mueller polarimeter. The calibration assumes a vertical polarized light source.
The polarimeter consists on a polarized light source, on a PSG a $\lambda/2$ followed by a $\lambda/4$, then after the sample a $\lambda/4$ followed by a linear polarizer.

The **initial calibration** consists on finding the steps per revolution or the steps per degree of each motor.
Later the elements should be positioned as follows: Linear Polarizer, $\lambda/2 (PSG)$, $\lambda/4 (PGS)$, $\lambda/4 (PSA)$ following the commands regarding the **second callibration**.


In [5]:
from polarimeter_functions import *
import timeit
import serial
# import tisgrabber as IC
import numpy as np
import pandas as pd
import cv2 as cv

In [6]:
def measure_irradiance(Camera):
    Camera.SnapImage()
    image = Camera.GetImage()
    image = cv2.flip(image, 0)
    total_irradiance = np.sum(image)
    return total_irradiance

# Function to move motor
def move_motor(ser, motor_number, steps):
    command = f"{motor_number},{steps}\n"
    ser.write(command.encode())

def calibration_spr(ser,cam,motor_number, step,iterations):
    irradiance_ = []
    position_ = []
    time_ = []
    position = 0
    time0 = timeit.timeit()
    for it in range(iterations):
        move_motor(ser,motor_number, step)
        irradiance = measure_irradiance(cam)
        time = timeit.timeit()
        time_.append(time)
        # print(irradiance)
        irradiance_.append(irradiance)
        position = position + step
        position_.append(position)
    df = pd.DataFrame({'Position': position_,'Irradiance':irradiance_, 'Time':time_})
    df.to_csv(f'Calibration_motor_{motor_number}.csv')
    return df

def steps_per_degree(df, motor_number):
    df.Irradiance = (df.Irradiance / df.Irradiance.max()) * 2 -1
    df.Irradiance = cv.GaussianBlur(np.array(df.Irradiance.values), ksize=(0, 0), sigmaX=50, borderType=cv.BORDER_REPLICATE)
    change_indexes = np.where(np.diff(np.sign(df.Irradiance)))[0] + 1
    change_positions = df.Position.iloc[change_indexes]
    position_diffs = np.diff(change_positions)
    average_position_diff = np.mean(position_diffs)

    max_freq = np.pi / average_position_diff
    fitted_curve = 0.5 * np.sin( (max_freq) * df['Position'] + 1500)
    fig = go.Figure()
    # Scatter plot for the original data
    fig.add_trace(go.Scatter(x=df.Position, y=df['Irradiance'], mode='markers', name='Original Data'))
        # Line plot for the fitted curve
    fig.add_trace(go.Scatter(x=df.Position, y=fitted_curve, mode='lines', name='Fitted Curve', line=dict(color='red')))

    # Set axis labels and plot title
    fig.update_layout(title=f'Fitted Sinusoidal Curve motor {motor_number}',
                      xaxis_title='Position',
                      yaxis_title='Irradiance')

    # Show the interactive plot
    fig.write_html(f'Malus_Motor_{motor_number}.html')
    print(f'The motor takes {2*average_position_diff} steps to run 360 degrees. Taking {2*average_position_diff/360} steps to move 1 degree')
    return 2*average_position_diff/360

# Function to calibrate the polarizer
def calibrate_element(ser,cam,motor_number,spd):

    initial_position = 0

    # Measure irradiance
    initial_irradiance = measure_irradiance(cam)

    # Initialize variables to track minimum intensity and corresponding position
    min_intensity = initial_irradiance
    min_intensity_position = initial_position
    steps = 0
    # Move the motor in steps until the intensity starts increasing
    while True:
        move_motor(ser,motor_number, spd)
        current_irradiance = measure_irradiance(cam)

        if current_irradiance > min_intensity:
            # Intensity has started increasing, break the loop
            break

    # Move the motor back to the position with minimum intensity
    move_motor(ser, motor_number, spd*359)

    # Optionally, you can measure the intensity at this position
    final_irradiance = measure_irradiance(cam)

    print(f"Calibration complete. Initial Irradiance: {initial_irradiance}, Minimum Irradiance: {min_intensity}, Final Irradiance: {final_irradiance}")


def mueller_mat(data):
    m00 = data['HH'] + data['HV'] + data['VH'] + data['VV']
    # m00 = m00 / m00.mean()
    m02 = data['PH'] + data['PV'] - data['MH'] - data['MV']
    # m02 = m02 / m02.mean()
    m10 = data['HH'] - data['HV'] + data['VH'] - data['VV']
    # m10 = m10 / m10.mean()
    m12 = data['PH'] - data['PV'] - data['MH'] + data['MV']
    # m12 = m12 / m12.mean()
    m20 = data['HP'] - data['HM'] + data['VP'] - data['VM']
    # m20 = m20 / m20.mean()
    m22 = data['PP'] - data['PM'] - data['MP'] + data['MM']
    # m22 = m22 / m22.mean()
    m30 = data['HR'] - data['HL'] + data['VR'] - data['VL']
    # m30 = m30 / m30.mean()
    m32 = data['PR'] - data['PL'] - data['MR'] + data['ML']
    # m32 = m32 / m32.mean()
    m01 = data['HH'] + data['HV'] - data['VH'] - data['VV']
    # m01 = m01 / m01.mean()
    m03 = data['RH'] + data['RV'] - data['LH'] - data['LV']
    # m03 = m03 / m03.mean()
    m11 = data['HH'] - data['HV'] - data['VH'] + data['VV']
    # m11 = m11 / m11.mean()
    m13 = data['RH'] - data['RV'] - data['LH'] + data['LV']
    # m13 = m13 / m13.mean()
    m21 = data['HP'] - data['HM'] - data['VP'] + data['VM']
    # m21 = m21 / m21.mean()
    m23 = data['RP'] - data['RM'] - data['LP'] + data['LM']
    # m23 = m23 / m23.mean()
    m31 = data['HR'] - data['HL'] - data['VR'] + data['VL']
    # m31 = m31 / m
    # 31.mean()
    m33 = data['LL'] - data['RL'] - data['LR'] + data['RR']
    # m33 = m33 / m33.mean()
    return m00, m01, m02, m03, m10, m12, m11, m13, m21, m20, m23, m30, m31, m32, m33, m22

## *Default parameters*

In [7]:
step = 50
iterations = 3000

# Initialize serial port
ser = serial.Serial('COM8', 115200)

SerialException: could not open port 'COM8': OSError(22, 'The semaphore timeout period has expired.', None, 121)

In [ ]:
move_motor(ser,1, 50)

In [ ]:
# Initialize camera
Camera = IC.TIS_CAM()
Camera.open('DMx 41BU02 8410421')
Camera.StartLive(1)
Camera.SnapImage()
measure_irradiance(Camera)

# Calibration 1: *Calibration of the steps required to move 1 degree.*

## Calibration 1, MOTOR 1

In [ ]:
motor_1 = calibration_spr(ser,Camera,1,step,1000)
# motor_1 = pd.read_csv('Calibration_motor_1.csv',index_col=0)
d_m1 = steps_per_degree(motor_1,1)

## Calibration 1, MOTOR 2

In [ ]:
move_motor(ser,3,200)

In [ ]:
motor_2 = calibration_spr(ser,Camera,1,step,1000)
# motor_2 = pd.read_csv('Calibration_motor_2.csv',index_col=0)
d_m2 = steps_per_degree(motor_2,2)

## Calibration 1, MOTOR 3

In [ ]:
# motor_3 = calibration_spr(Camera,1,step,iterations)
motor_3 = pd.read_csv('Calibration_motor_3.csv',index_col=0)
d_m3 = steps_per_degree(motor_3,3)

## Calibration 1, MOTOR 4

In [ ]:
ser.close()
ser = serial.Serial('COM8', 115200)
motor_4 = calibration_spr(ser,Camera,4,step,iterations)
# motor_4 = pd.read_csv('Calibration_motor_4.csv',index_col=0)
d_m4 = steps_per_degree(motor_4,4)

# Calibration 2: *Finding the zero or fast axis of each of the polarization elements*

 Linear polarizer PSA located at **90 degrees**

In [ ]:
calibrate_element(Camera,motor_4,d_m4)


*PSG* $\lambda /2$'s fast axis aligned to the polarization axis of the source, *zero degrees*


In [ ]:
calibrate_element(Camera,motor_1,d_m1)

*PSA* $\lambda /4$'s fast axis aligned to the polarization axis of the source, *zero degrees*


In [ ]:
calibrate_element(Camera,motor_2,d_m2)

*PSA* $\lambda /4$'s fast axis aligned to the polarization axis of the source, *zero degrees*

In [ ]:
calibrate_element(Camera,motor_3,d_m3)

The intensity should remain at zero, then the linear polarized is shifted again to its zero.

In [ ]:
measure_irradiance(Camera)
move_motor(motor_4, d_m4 * 90)
measure_irradiance(Camera)

# Mueller matrix measurement

## Initial development just for bright field


In [ ]:
save_path = 'C:\Users\mlope\OneDrive - Universidad EAFIT\EAFIT\Polarimeter\Polarimeter\Data\CalibrationTestDP'
angles = np.array([[0, 0, 90, 90], [0, 0, 90, 0], [0, 0, -45, 45], [0, 0, 45, -45],[0, 0, 0, -45], [0, 0, 0, 45],
                   [45, 90, 90, 90], [45, 90, 90, 0], [45, 90, -45, 45],[45, 90, 45, -45], [45, 90, 0, -45], [45, 90, 0, 45],
                   [-22.5, -45, 90, 90],[-22.5, -45, 90, 0], [-22.5, -45, -45, 45], [-22.5, -45, 45, -45], [-22.5, -45, 0, -45],[-22.5, -45, 0, 45],
                   [22.5, 45, 90, 90], [22.5, 45, 90, 0], [22.5, 45, -45, 45], [22.5, 45, 45, -45], [22.5, 45, 0, -45], [22.5, 45, 0, 45],
                   [22.5, 90, 90, 90],[22.5, 90, 90, 0], [22.5, 90, -45, 45], [22.5, 90, 45, -45], [22.5, 90, 0, -45], [22.5, 90, 0, 45],
                   [45, 90, 90, 90], [45, 90, 90, 0], [45, 90, -45, 45], [45, 90, 45, -45], [45, 90, 0, -45], [45, 90, 0, 45], [0, 0, 0, 0]])


df = pd.DataFrame(columns=['Name','Counts'])
data = {}
names = [['HV', 'HH','HP','HM','HR','HL',
          'VV','VH','VP','VM','VR','VL',
          'MV','MH','MP','MM','MR','ML',
          'PV','PH','PP','PM','PR','PL',
          'LV','LH','LP','LM','LR','LL',
          'VV','VH','VP','VM','VR','VL','ref']]

for angle, name in zip(angles,names):
    move_motor(ser, motor_4, d_m4 * angle[0])
    move_motor(ser, motor_4, d_m4 * angle[1])
    move_motor(ser, motor_4, d_m4 * angle[2])
    move_motor(ser, motor_4, d_m4 * angle[3])

    time.sleep(1)
    image  = Camera.GetImage()
    data[name] = image # directly just for bright field
    cv.imwrite(image,f'{save_path}/{name}.png')
    df = df.append({'Name':name,'Counts':measure_irradiance(Camera)},ignore_index = True)

m00, m01, m02, m03, m10, m12, m11, m13, m21, m20, m23, m30, m31, m32, m33, m22 = mueller_mat(data)